In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
import re
from sus import badwords,symbols

In [2]:
# req_bad = pd.read_csv('./datasets/2bad_reqff.csv')
# print(req_bad.head())

In [3]:
# req_good = pd.read_csv('./datasets/2good_reqff.csv')
# print(req_good.head())

In [7]:
feature_columns = [
    'single_q', 'double_q', 'dashes', 'braces', 'spaces', 
    'percentages', 'semicolons', 'angle_brackets', 'special_chars', 
    'badwords_count','path_length', 'body_length'
]

pred_cols =  [
    'path_single_q', 'path_double_q', 'path_dashes', 'path_braces', 'path_spaces', 
    'path_percentages', 'path_semicolons', 'path_angle_brackets', 'path_special_chars', 
    'path_badwords_count', 'body_single_q', 'body_double_q', 'body_dashes', 'body_braces', 
    'body_spaces', 'body_percentages', 'body_semicolons', 'body_angle_brackets', 
    'body_special_chars', 'body_badwords_count', 'path_length', 'body_length'
]


In [14]:
all_data = pd.read_csv('./datasets/all_datas_f.csv')


In [ ]:
all_data

In [15]:
all_data.drop(columns=feature_columns,inplace=True)
all_data.fillna("",inplace=True)


In [9]:
def count_symbols(text, symbols):
    if pd.isna(text):
        return 0
    return sum(text.count(symbol) for symbol in symbols)

def count_bad_words(text, badwords):
    if pd.isna(text):
        return 0
    return sum(1 for word in badwords if word in text.lower())

In [16]:
for col in ['path', 'body']:
    all_data[f'{col}_single_q'] = all_data[col].apply(lambda x: count_symbols(x, ["'"]))
    all_data[f'{col}_double_q'] = all_data[col].apply(lambda x: count_symbols(x, ['"']))
    all_data[f'{col}_dashes'] = all_data[col].apply(lambda x: count_symbols(x, ["--"]))
    all_data[f'{col}_braces'] = all_data[col].apply(lambda x: count_symbols(x, ["{", "}"]))
    all_data[f'{col}_spaces'] = all_data[col].apply(lambda x: count_symbols(x, [" "]))
    all_data[f'{col}_percentages'] = all_data[col].apply(lambda x: count_symbols(x, ["%"]))
    all_data[f'{col}_semicolons'] = all_data[col].apply(lambda x: count_symbols(x, [";"]))
    all_data[f'{col}_angle_brackets'] = all_data[col].apply(lambda x: count_symbols(x, ["<", ">"]))
    all_data[f'{col}_special_chars'] = all_data[col].apply(lambda x: sum(1 for c in str(x) if not c.isalnum() and not c.isspace()) if pd.notna(x) else 0)
    all_data[f'{col}_badwords_count'] = all_data[col].apply(lambda x: count_bad_words(x, badwords))

# General features based on length


In [11]:
all_data['path_length'] = all_data['path'].apply(lambda x: len(x) if pd.notna(x) else 0)
all_data['body_length'] = all_data['body'].apply(lambda x: len(x) if pd.notna(x) else 0)

In [12]:
all_data

  method                                               path  \
0   POST                                           /doLogin   
1   POST                                      /sendFeedback   
2    GET                                 /admin/clients.xls   
3    GET  /my%20documents/JohnSmith/Bank%20Site%20Docume...   
4    GET  /my%20documents/JohnSmith/Bank%20Site%20Docume...   

                                                body  class  path_single_q  \
0                  uid=ZAP&passw=ZAP&btnSubmit=Login      1              0   
1  cfile=comments.txt&name=ZAP&email_addr=ZAP&sub...      1              0   
2                                                         1              0   
3                                                         1              0   
4                                                         1              0   

   path_double_q  path_dashes  path_braces  path_spaces  path_percentages  \
0              0            0            0            0                 0  

In [17]:
X = all_data[feature_columns]
y = all_data['class']

KeyError: "None of [Index(['single_q', 'double_q', 'dashes', 'braces', 'spaces', 'percentages',\n       'semicolons', 'angle_brackets', 'special_chars', 'badwords_count',\n       'path_length', 'body_length'],\n      dtype='object')] are in the [columns]"

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [42]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))


Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.99      0.97        80
           1       0.99      0.95      0.97        77

    accuracy                           0.97       157
   macro avg       0.97      0.97      0.97       157
weighted avg       0.97      0.97      0.97       157



In [37]:
joblib.dump(model, 'request_threat_model.pkl')
print("Model saved as 'request_threat_model.pkl'")

Model saved as 'request_threat_model.pkl'
